<a href="https://colab.research.google.com/github/KaustubhRatna/Deep-Learning-Projects/blob/main/bellpepper_ResNet50_withoutGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom arch 

##Library and dataset importing

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

In [ ]:
#constants
IMAGE_SIZE=256
BATCH_SIZE = 64
CHANNELS =3
EPOCHS = 10
FILE_PATH="DiabeticRetinopathy"
import os
os.environ["KAGGLE_CONFIG_DIR"]= "/usr/local/bin" #folder/usr/local/bin/kaggle

In [ ]:
!kaggle competitions download -c diabetic-retinopathy-detection

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.8/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.8/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /usr/local/bin. Or use the environment method.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

!unzip \*.zip && rm *.zip

unzip:  cannot find or open *.zip, *.zip.zip or *.zip.ZIP.

No zipfiles found.


In [ ]:
!cp -r "/content/PlantVillage/Pepper__bell___Bacterial_spot" "/content/BellPepper"
!cp -r "/content/PlantVillage/Pepper__bell___healthy" "/content/BellPepper"

cp: cannot stat '/content/PlantVillage/Pepper__bell___Bacterial_spot': No such file or directory
cp: cannot stat '/content/PlantVillage/Pepper__bell___healthy': No such file or directory


In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
            directory='/content/BellPepper',
            shuffle=True,
            image_size=(IMAGE_SIZE,IMAGE_SIZE),
            batch_size = BATCH_SIZE,
)

NotFoundError: ignored

In [ ]:
class_names = dataset.class_names
class_names

##Data Visualization

In [ ]:
plt.figure(figsize=(20,10))
for image_batch, label_batch in dataset.take(1):
    for i in range(15):
        ax = plt.subplot(3,5,i+1)
        plt.imshow(image_batch[i].numpy().astype('uint8'))
        plt.title(class_names[label_batch[i]])
        plt.axis("off")

##Train Test Split

In [ ]:
def get_dataset_partitions(ds,train_split=0.8,test_split=0.1,valid_split=0.1,shuffle=True,shuffle_size=10000):
    assert  train_split+test_split+valid_split==1
    ds_size = len(ds)
    if shuffle:
        ds.shuffle(shuffle_size,seed=12)
    train_size = int(len(ds)*train_split)
    valid_size = int(len(ds)*valid_split)

  
    train_ds = ds.take(train_size)    
    valid_ds = ds.skip(train_size).take(valid_size)
    test_ds = ds.skip(train_size).skip(valid_size)
    
    return train_ds ,test_ds, valid_ds
    

In [ ]:
train_ds ,test_ds, valid_ds = get_dataset_partitions(dataset)
    

In [ ]:
print(len(train_ds))
print(len(test_ds))
print(len(valid_ds))

In [ ]:
# performance improving by both using CPU and GPU
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
valid_ds = valid_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

##Image Preprocessing

In [ ]:
rescale_and_resize=tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE) ,# will resize to the IMAGE_SIZE if there's any issue
    layers.experimental.preprocessing.Rescaling(1./255)  # Normalization
])

In [ ]:
#data augmentation

data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2)
])

## Model training

In [ ]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 2


model = models.Sequential([
    rescale_and_resize,
    data_augmentation,
    
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='sigmoid'),
])

model.build(input_shape=input_shape)

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"]
)

In [ ]:
#model training
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=valid_ds,
    verbose=1,
    epochs=EPOCHS
    
)

In [ ]:
scores = model.evaluate(test_ds)
scores

#ResNet50

In [ ]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 2

resnet_model = tf.keras.applications.ResNet50(
    input_shape=(256,256,3), 
    include_top=False, 
    weights='imagenet'
)

resnet_model.trainable =False

resnet_model.summary()


94765736/94765736 [==============================] - 4s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                          

In [ ]:
resnettrain = tf.keras.Sequential([
    resnet_model,
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(2, activation='sigmoid')
])

resnettrain.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 batch_normalization_10 (Bat  (None, 8, 8, 2048)       8192      
 chNormalization)                                                
                                                                 
 flatten_5 (Flatten)         (None, 131072)            0         
                                                                 
 dense_12 (Dense)            (None, 16)                2097168   
                                                                 
 dropout_10 (Dropout)        (None, 16)                0         
                                                                 
 dense_13 (Dense)            (None, 8)                 136       
                                                      

In [ ]:
resnettrain.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"]
)

In [ ]:
history = resnettrain.fit(
    train_ds,
    batch_size=128,
    validation_data=valid_ds,
    verbose=1,
    epochs=10
    
)

Epoch 1/10
19/19 [==============================] - 269s 14s/step - loss: 0.4894 - accuracy: 0.8520 - val_loss: 0.4031 - val_accuracy: 1.0000
Epoch 2/10
19/19 [==============================] - 268s 14s/step - loss: 0.4467 - accuracy: 0.8791 - val_loss: 0.3755 - val_accuracy: 1.0000
Epoch 3/10
19/19 [==============================] - 267s 14s/step - loss: 0.4106 - accuracy: 0.9153 - val_loss: 0.3338 - val_accuracy: 1.0000
Epoch 4/10
19/19 [==============================] - 266s 14s/step - loss: 0.3774 - accuracy: 0.9383 - val_loss: 0.3007 - val_accuracy: 1.0000
Epoch 5/10
19/19 [==============================] - 268s 14s/step - loss: 0.3422 - accuracy: 0.9572 - val_loss: 0.2732 - val_accuracy: 1.0000
Epoch 6/10
19/19 [==============================] - 267s 14s/step - loss: 0.2984 - accuracy: 0.9523 - val_loss: 0.2380 - val_accuracy: 1.0000
Epoch 7/10
19/19 [==============================] - 268s 14s/step - loss: 0.2718 - accuracy: 0.9663 - val_loss: 0.2088 - val_accuracy: 1.0000
Epoch 